### Token和Langchain总览
#### Token的概念
在开始Langchain之前，先补充一个重要的概念：Token。相信大家平时已经注意到了，不管是ChatGPT还是其他什么模型，只要是收费的服务，它的计费单位一般都是每1000token多少钱。而语言模型一般来说都有个上下文大小，比如ChatGPT是4096，GPT4默认是8K，最大可到32K。这里的单位也是token。比如使用ChatGPT时，你的问题和回答之和最多只能有4096个token。

在之前分享wolfram的《这就是ChatGPT》这本书的时候曾经说过NLP模型的基本原理都是根据语料库的概率一次预测一个词：
  
<center>
  <img src="./images/token.png" width="1200">
</center>

其实这里的词严格来说就是token。ChatGPT的官方提供了[一个网站](https://platform.openai.com/tokenizer)可以可视化token

<center>
  <img src="./images/tokenizer.png" width="1200">
</center>

我们可以通过下面的程序来证明，语言模型确实是一个token一个token输出的。这里使用了langchain的流式输出，每次for循环就表示LLM进行了一次输出，我们将每次的输出间隔符写成`|`，然后和上图的Tokenizer结果对比就会发现每次输出刚好是对应一个token。因此在transformer中，token是最小的输出单位。

In [1]:
# Token.ts
from langchain_community.llms import Ollama

llm = Ollama(model='qwen:14b')
for chunk in llm.stream("hello"):
    print(chunk, end='|', flush=True)

Hello| there|!| How| may| I| assist| you| today|?||

那么为什么语言模型有最大的token限制呢。这主要是因为几个原因：
1. 模型规模和token数量在正常情况下是线性关系的，处理token越多，计算开销越大
2. 输入和中间结果需要存放在内存中，token过大会导致占用更多内存
3. 输入太大会反而影响推理的效率和准确性

#### langchain
上次分享结尾已经提过langchain，应该都知道它是一个LLM应用开发框架。一般来说，我们把一个库能称为框架，肯定是因为它做了某种抽象，提出了某种开发的模型。拿大家比较熟悉的前端的例子，在前端早期，那时候基本都在使用jQuery。我们很少将jQuery称为一个框架，是因为它虽然封装了很多很有用的函数，但是它的思维方式是和用原生操作DOM的思路是一样的，没有进行任何抽象。而后来的vue，react等，都有自己的模型和抽象，因此我们都把这些称为框架。

同样的langchain虽然内置了很多工具方法，这些方法不用langchain你自己也可以去实现，但是它提出了一个自己的抽象，即所谓的chain，或者Runnable接口，这个是我们后面分享的一个重点，也是langchain的一个核心思想，在此之前我们先来总览下langchain的各个部分：

<center>
  <img src="./images/langchain.png">
</center>

上图是从官网截取的一个langchain的生态架构图，可以看出langchain的生态主要分为四块：

##### LangSmith
这是langchain提供的调试，监控，追踪，分析，评估LLM应用的一个开发平台，可以免费限量使用，没有开放源码，私有部署需要付费，普通用户可以免费限额使用。建议在自己学习langchain的时候都把这个对接上，可以让你观察到很多关于chain的中间状态，让你更清晰地知道你的整个链路的运行中间过程。

##### LangServe
这个是langchain提供的简化LLM部署的工具，只提供了python版本。个人感觉相比之下是没有其他组件重要，毕竟开发完成之后封装成Restful接口即使没有这个，使用python的一些web框架也可以很容易做到，当然它也提供了一些有意思的方便的功能。

##### Template
它其实就是个脚手架模板的市场，上面有很多官方或者社区分享的可以快速搭建一个结构的脚手架，比如你可以选择一个openai+rag的Template，快速生成一个使用ChatGPT的RAG应用框架。

##### Langchain
这是Langchain的核心。从图上也可以看出，它也大概可以分为Langchain，Langchain_Community和Langchain_Core三个部分。官方维护了python和js两个版本。其中js版本可以用于浏览器和node环境，主要是提供给前端开发者使用。当然社区也有其他语言版本，java，golang，rust啥的，但是这些基本都是个人维护，没有官方的稳定，如果要使用建议慎重。
开始详细介绍Langchain前要吐槽一点，就是个人学习Langchain过程中觉得最难受的就是这几个包各自承载了什么功能很乱。虽然LLM应用开发本身确实是一个新的技术领域，所以langchain这种框架的api一直在与时俱进的调整是很正常的，但是在使用过程中你会发现某个模型在langchain下，某个模型在langchain_community下，甚至某些模型还有单独的包，就非常的混乱。加上python和js本来作为一个弱类型语言，提示也没有强类型语言那么准确，因此在学习过程中，多查官方文档是很有必要的。

#### Langchain的六大模块
##### Model IO
这一部分可以细化为三块，第一就是PromptTemplate，也就是提示词模板，负责以模板形式生成提示词，同时也有一些内置的提示词。然后就是LLM，这一块主要就是负责和语言模型进行交互，调用语言模型。第三部分Output Parser，这部分主要是用来控制LLM的输出，比如我希望LLM的输出不是文字，而是json，或者一个时间类型，这样便于我后面程序的对接，这时候就需要使用Output Parser。

##### Retrieval
这一块主要和RAG相关，可以再细分为六个部分：

第一就是Document Loaders，顾名思义，就是你做RAG的时候肯定会有一些外部数据需要程序处理，而这种数据的来源可以说各种各样，比如从PDF，Word等文件读取，又或者在某个网页上的一个元素中的内容，又或者是某个视频网站上的某个视频的弹幕等等。而Document Loaders提供了统一的抽象并提供了各种各样的加载器支持从各种源加载数据。

第二就是Text Splitters。在前面提到过，LLM都有个窗口大小，上下文不是无限大的，因此对于我们加载进来的文档在写入向量库之前，是需要进行文档分割的。同时文档越大，在向量化后失去的信息也就越多。同时检索的相关性也会降低。比如一整本书只是提到了python，其他都是和python无关的内容。但是在用户询问python相关的问题的时候却召回了，这明显会大大影响结果的正确和稳定性。

第三就是Embedding Models。要对分割好的字符进行向量化，肯定需要调用一个embedding模型，类似Model IO中的LLM部分，负责和模型交互

第四就是Vector Stores。从名字就知道，通过这个组件可以和各种向量数据库进行交互。

第五个是Retrievers。这个主要是当用户询问一个问题的时候，如何从向量数据库召回哪些文档片段的策略，如何判断向量库中哪些数据和问题相关。

最后一个是索引，这个和关系型数据库的索引其实类似，避免你重复写入文档片段到向量库，可以避免重复计算已经计算过的向量值等等。

##### Chain
langchain的核心概念，本质是将各种组件组合起来的一种机构，底层是一个抽象类，后面会具体重点介绍以及说明原理。

##### Memory
LLM没有存储机构，因此在连续对话过程中其实并不知道上下文。那么为啥我们和语言模型对话的时候发现它似乎记得前面说过啥。其实最早观察过ChatGPT的接口的应该都会发现每次对话都将之前对话历史附带上了，因此模型知道之前的历史，这就是一种memory。当然现在再去看会发现请求中没有具体的历史对话内容了，取而代之的是一个对话编号。当然Memory有很多类型，后面会具体介绍。

##### Agent
Agent在langchain里面被翻译成代理，它还有个常见的意思就是智能体。说到Agent大家都会觉得很神秘，觉得很高级，但是都说不清楚Agent是啥，因为这个东西大家都还在探索中。Langchain中的Agent采用了ReAct的框架，增强了LLM的推理能力和行动能力，我们会在后面写一个简单的Agent，但是我们暂时不能给Agent下一个清晰的定义。

##### Tool
在昨年下半年OpenAI给ChatGPT解锁了函数调用的能力，这就好像一个机器人之前虽然有一个很强的芯片，有各种精度很高的传感器，但是没有执行机构，只能给你打字。在可以调用函数之后LLM可以根据你预设的描述在它认为合适的时候调用合适的方法。这个一般和Agent一起用于构建所谓的智能体。Tool定义各种可以给LLM调用的操作以及操作的描述。而Agent背后的调度器负责分析什么时候应该调用哪个函数。

以上就是Langchain的六大组件，其实应该来说还有一个叫CallBack的组件，这个组件主要是一个hook系统，你通过callback可以完成对Langchain运行各个部分的埋点，监控等，和开发的关系不太大，这里就忽略了有兴趣可以去官网文档了解。在准备这次分享的过程中，Langchain也升级到了0.2.x，因此这次讲的有些东西比如api调用方式，memory等都在0.2.x版本发生了变化。Langchain本来就算是大家目前对LLM应用开发的一些范式的抽象和总结。而LLM开发本来也就只是个一年多的新领域，势必在不断更新和变化中，因此Langchain也会一直不断在变化，所以在学习Langchain过程中，重要的不是将API记得非常清楚，因为很多，很乱。重要的是将各个组件的作用，背后的思想弄明白，这样哪怕Langchain再升级，甚至你不使用Langchain，你都有写一个LLM应用的思路。
